In [5]:
import cv2
import os
import shutil
import numpy as np

In [13]:
# Function to create a folder
def createFolder(name):
    try:
        os.mkdir(name)
        print('Folder created!')
    except:
        print('Folder already exists!')

# Create opFolder
def createOpFolder(vid, opFolder):
    # If the video file is not found or some other error occured in opening the file
    if not vid.isOpened():
        print('Error while opening the file')
    # If the file is opened, create an output folder to save frames of the video
    else:
        # If ouput folder exists, delete its data
        if os.path.isdir(opFolder):
            try:
                # Delete existing output folder and create a new one
                shutil.rmtree(opFolder)
                print('Old folder deleted')
                createFolder(opFolder)
            except:
                print('Old folder not deleted')
        else:
            createFolder(opFolder)
        
def convertVidToIm(filename, opFolder):
    vid = cv2.VideoCapture(filename)

    createOpFolder(vid, opFolder)

    # Frames array
    opFrames = []
    frameCount = 0
    while(vid.isOpened()):
        # Return is True if the read() returns a frame
        # frame is the frame of the video
        ret, frame = vid.read()

        # If ret is True, show the frame
        if ret:
            frameCount = frameCount + 1
            opFrames.append(frame)        
            cv2.imshow('frame',frame)

            # Save frame in output folder
            fn = opFolder + '/%d.jpg'%frameCount
            cv2.imwrite(fn, frame)

            # Press 'q' to quit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break


    opFrames = np.asarray(opFrames)
    
    vid.release()
    cv2.destroyAllWindows()
    return opFrames

def playVideo(filename):
    vid = cv2.VideoCapture(filename)

    while(vid.isOpened()):
        # Return is True if the read() returns a frame
        # frame is the frame of the video
        ret, frame = vid.read()

        # If ret is True, add the frame
        if ret:
            cv2.imshow('frame',frame)

            # Press 'q' to quit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break
 
    vid.release()
    cv2.destroyAllWindows()

# Converts a list of images into a single video
def convertImToVid(images, fps, frameSize):
    name = 'outputVideo.mp4'
    opvid = cv2.VideoWriter(name, cv2.VideoWriter_fourcc('M','J','P','G'), fps, frameSize, True)
    for im in images:
        opvid.write(im)
    
    opvid.release()
    cv2.destroyAllWindows()
    
    playVideo(name)
    
# fgVid = Foreground video
# bgVid = Background video
def chromaKeying(fgVid, bgVid):
    fgFolder = 'fg'
    bgFolder = 'bg'
    fgFrames = convertVidToIm(fgVid, fgFolder)
    bgFrames = convertVidToIm(bgVid, bgFolder)
    
    if not ((len(fgFrames) == 0) & (len(bgFrames) == 0)):
        print(len(fgFrames))
        print(len(bgFrames))
        
        A = 1.0
        B = 1.0    
        newFgFrames = []

        fs = fgFrames[0].shape

        for fg, bg in zip(fgFrames, bgFrames):

            # For each pixel, check whether fg or bg
            for i in range(0, fs[0]):
                for j in range(0,fs[1]):
                    fgVal = fg[i,j]
                    alpha = np.int16(fgVal[0]) + np.int16(fgVal[2]) - np.int16(fgVal[1])
#                     alpha = ((A * (fgVal[0] + fgVal[2])) - (B * fgVal[1]))
#                     print(alpha)

                    # Replace with background pixel value if alpha is less than 0
                    if alpha <= 0:
                        fg[i,j] = bg[i,j]

                    # Remove green spill
                    fg[i,j] = [fgVal[0], min(fgVal[1], fgVal[2]), fgVal[2]]

            newFgFrames.append(fg)

        convertImToVid(newFgFrames, 25, (fs[1], fs[0]))
    

In [15]:
# Constants
fg = 'fgVid.mp4'
bg = 'bgVid.mp4'

chromaKeying(fg, bg)

Old folder deleted
Folder created!
Old folder deleted
Folder created!
398
894
